In [2]:
#Loading the data
from langchain_community.document_loaders import WebBaseLoader

In [3]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
data = loader.load()
data

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly unders

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [5]:
#split the text in the documnets into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [7]:
len(documents)

4

In [9]:
from langchain_community.embeddings import OllamaEmbeddings
## FAISS Vector Database
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OllamaEmbeddings())

In [10]:
db

In [11]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [12]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [14]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000181F22F8550>)

In [15]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [16]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [17]:
response['answer']

'Based on the provided context, the answer to the question "Scaled Dot-Product Attention" is:\n\nLangSmith supports scaling dot-product attention as part of its workflow for LLM application development, monitoring, and testing.\n\nIn the context of LangSmith, scaled dot-product attention refers to a technique used to modify the dot-product attention mechanism to improve the performance of the language model. The basic idea behind scaled dot-product attention is to multiply the dot product of the query and key vectors by a scalar value before computing the attention weights. This allows the model to focus more on the most important parts of the input sequence and reduce the impact of irrelevant information.\n\nLangSmith provides various features and tools for working with scaled dot-product attention, including:\n\n1. Annotating traces: LangSmith allows users to annotate traces with respect to different criteria, which can help identify areas where scaled dot-product attention is needed